In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/persian-quotes/data.json


In [2]:
train = pd.read_json('../input/persian-quotes/data.json', encoding='utf8')
train.head()

,body,author
0,بشریت باید جنگ را ریشه کن کند، پیش از آنکه جنگ...,جان اف کندی
1,هیچ چیز ترسناک‌‌تر و شرم‌آورتر از این نیست که ...,افلاطون
2,کسی که روح شادی دارد، همیشه وسیله خوبی برای شا...,آرتور شوپنهاور
3,روزی یک آتنی نزد پیشگوی معبد دلفی رفت و از آپو...,یاستین گوردر
4,« تو نخواهی کشت» تنها درباره کشتن همنوع صدق نم...,لئو تولستوی


In [3]:
train.isnull().sum()

body      0
author    0
dtype: int64

In [4]:
#Code by Ion Mosnoi https://www.kaggle.com/mosnoiion/countvectorizer-multinomialnb

#We don't have test/submission. Only train/df.

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(train['body'])
X_test_counts = count_vect.transform(train['author'])

tf_transformer = TfidfTransformer(use_idf=True).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_test = tf_transformer.transform(X_test_counts)


X_train_counts.shape

(739, 5713)

#Create a label column

In [5]:
#Code by Ion Mosnoi https://www.kaggle.com/mosnoiion/countvectorizer-multinomialnb

labels = train['author'].unique().tolist()
train['label'] = train['author'].apply(lambda x:labels.index(x))
train.head()

,body,author,label
0,بشریت باید جنگ را ریشه کن کند، پیش از آنکه جنگ...,جان اف کندی,0
1,هیچ چیز ترسناک‌‌تر و شرم‌آورتر از این نیست که ...,افلاطون,1
2,کسی که روح شادی دارد، همیشه وسیله خوبی برای شا...,آرتور شوپنهاور,2
3,روزی یک آتنی نزد پیشگوی معبد دلفی رفت و از آپو...,یاستین گوردر,3
4,« تو نخواهی کشت» تنها درباره کشتن همنوع صدق نم...,لئو تولستوی,4


In [6]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tf, train['label'])

In [7]:
pred = clf.predict(X_train_test)
pred.shape

(739,)

In [8]:
submission = train
submission['label'] = pred

In [9]:
submission['author'] = submission['label'].apply(lambda x: labels[x])
submission.head()

,body,author,label
0,بشریت باید جنگ را ریشه کن کند، پیش از آنکه جنگ...,فردریش نیچه,19
1,هیچ چیز ترسناک‌‌تر و شرم‌آورتر از این نیست که ...,فردریش نیچه,19
2,کسی که روح شادی دارد، همیشه وسیله خوبی برای شا...,فردریش نیچه,19
3,روزی یک آتنی نزد پیشگوی معبد دلفی رفت و از آپو...,فردریش نیچه,19
4,« تو نخواهی کشت» تنها درباره کشتن همنوع صدق نم...,فردریش نیچه,19
